**Задание 3.1**

Найдите третий корень полинома $f(x) = 6x^5 - 5x^4 - 4x^3 +3x^2$, взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

In [1]:
def func_x(x):
    return 6*x**5 - 5*x**4 - 4*x**3 +3*x**2

def diff_x(x):
    return 30*x**4 - 20*x**3 - 12*x**2 + 6*x

In [5]:
x0 = 0.7
x_cur = x0
i = 1
while abs(func_x(x_cur)) > 0.00001:
    x_new = x_cur - func_x(x_cur)/diff_x(x_cur)
    print(i, ' ', round(x_cur, 3), ' ', round(x_new, 3), func_x(x_new))
    x_cur = x_new
    i += 1

1   0.7   0.63 -0.0012133284487552132
2   0.63   0.629 -1.3785387997788945e-06
